In [ ]:
# Check TensorFlow version
import tensorflow as tf
print(tf.__version__)

In [ ]:
# STEP 1: Install dependencies
!pip install -q tensorflow tensorflow_datasets

import tensorflow as tf
import tensorflow_datasets as tfds

print("TensorFlow version:", tf.__version__)

# STEP 2: Load the PlantVillage dataset
# (already available in TensorFlow Datasets)
(ds_train, ds_val), ds_info = tfds.load(
    'plant_village',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# STEP 3: Preprocess images (resize + normalize)
IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE)) / 255.0
    return image, label

train_ds = ds_train.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = ds_val.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# STEP 4: Build Transfer Learning Model (MobileNetV2)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(ds_info.features['label'].num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# STEP 5: Train model
history = model.fit(train_ds, validation_data=val_ds, epochs=5)

# STEP 6: Save and convert to TensorFlow Lite
model.save("plant_disease_model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Quantization for mobile
tflite_model = converter.convert()

with open("plant_disease_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Training done & TFLite model saved!")


In [ ]:
#after the training is done 
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="plant_disease_model.tflite") #here model name can be different
interpreter.allocate_tensors()

# Get input & output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load a test image (resize to model input size, e.g., 128x128)
img = Image.open("test_leaf.jpg").resize((128,128))
img_array = np.expand_dims(np.array(img)/255.0, axis=0).astype(np.float32)

# Run inference
interpreter.set_tensor(input_details[0]['index'], img_array)
interpreter.invoke()
prediction = interpreter.get_tensor(output_details[0]['index'])

print("Prediction:", prediction)


In [ ]:
#if you dont know your model name run this code it will tell you your files detail
!ls -lh

In [ ]:
#this is for uploading files and it will give you a choose button
from google.colab import files
uploaded = files.upload()

In [ ]:
#this is to check weather your model is working or not 
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="plant_disease_model.tflite")
interpreter.allocate_tensors()

# Get input & output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [ ]:
# Open and preprocess the image
img = Image.open("leaf.jpg").resize((224, 224))  # make sure size matches model input
img = np.array(img, dtype=np.float32) / 255.0   # normalize to 0-1
img = np.expand_dims(img, axis=0)               # add batch dimension

In [ ]:
# Run inference
interpreter.set_tensor(input_details[0]['index'], img)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

print("Raw output:", output_data)
print("Predicted class index:", np.argmax(output_data))

In [ ]:
#this will show you the image and the top 3 predictions with image
import matplotlib.pyplot as plt

# Get top 3 predictions
top_indices = np.argsort(output_data[0])[::-1][:3]

# Plot image + predictions
plt.figure(figsize=(6,6))
plt.imshow(img[0])   # remove batch dimension
plt.axis("off")
plt.title("Uploaded Leaf")

# Print results
print("=== Diagnostic Report ===")
for i in top_indices:
    print(f"{class_names[i]}: {output_data[0][i]*100:.2f}%")